# RAG（检索增强生成）系统实现

本 Notebook 演示如何使用 **LangChain + FAISS + 本地大模型**构建完整的 RAG 系统。

## 什么是 RAG？
RAG (Retrieval-Augmented Generation) 通过从知识库检索相关文档，将其作为上下文提供给大模型，从而生成更准确、更具针对性的回答。

## 实现步骤
1. **加载知识库文档**：从 `knowledge_base/` 目录读取文档
2. **文本分块**：使用 RecursiveCharacterTextSplitter 切分文档
3. **向量化**：使用 HuggingFace Embeddings 模型生成向量
4. **构建索引**：使用 FAISS 创建向量数据库
5. **检索测试**：测试相似度检索效果
6. **RAG 问答**：集成本地 Qwen 模型进行增强生成
7. **效果对比**：对比有/无 RAG 的回答差异

## 技术栈
- **LLM**: Qwen2.5-7B-Instruct-GPTQ-Int4
- **Embedding**: text2vec-base-chinese
- **向量数据库**: FAISS
- **框架**: LangChain

## 知识库内容
本示例包含以下私有数据：
- **班级花名册**：合肥工业大学宣城校区计算机科学与技术23-3班学生信息（50人）

**RAG 的核心价值**：让大模型能够回答关于私有数据的问题，这些信息是预训练数据中不存在的。


**示例场景**：- 查询班级特色信息（班干部、优秀学生等）

- 查询班级学生信息（姓名、学号、政治面貌）- 查询班级统计数据（男女比例、团员人数等）

## 1. 环境准备与依赖检查

In [4]:
import sys
import os

print(f"Python: {sys.version}")
print(f"工作目录: {os.getcwd()}")

# 检查必要的库
try:
    import langchain
    import faiss
    from sentence_transformers import SentenceTransformer
    print("✅ LangChain, FAISS, SentenceTransformers 已安装")
except ImportError as e:
    print(f"❌ 缺少依赖: {e}")
    print("请确保已安装 environment.yml 中的所有依赖")

Python: 3.10.19 | packaged by conda-forge | (main, Oct 22 2025, 22:23:22) [MSC v.1944 64 bit (AMD64)]
工作目录: d:\llm_deploy\LLM\LLM_DEPLOY
✅ LangChain, FAISS, SentenceTransformers 已安装


## 2. 加载知识库文档

从 `knowledge_base/sample_docs/` 目录加载所有文本文件。

In [43]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# 文档目录
docs_dir = './knowledge_base/sample_docs'

# 加载所有 .txt 文件
loader = DirectoryLoader(
    docs_dir,
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'}
)

documents = loader.load()

print(f"✅ 成功加载 {len(documents)} 个文档")
for i, doc in enumerate(documents):
    print(f"\n文档 {i+1}: {os.path.basename(doc.metadata['source'])}")
    print(f"  长度: {len(doc.page_content)} 字符")
    print(f"  预览: {doc.page_content[:100]}...")

✅ 成功加载 3 个文档

文档 1: class_roster_cs23-1.txt
  长度: 1526 字符
  预览: # 计算机科学与技术23-1班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-1班

## 班级特色
- 班长: 王天翔（学号：2023217101）...

文档 2: class_roster_cs23-2.txt
  长度: 1608 字符
  预览: # 计算机科学与技术23-2班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-2班

## 班级特色
- 班长: 陈浩宇（学号：2023217201）...

文档 3: class_roster_cs23-3.txt
  长度: 1598 字符
  预览: # 计算机科学与技术23-3班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-3班

## 班级特色
- 学习委员: 贺鑫帅（学号：202321737...


## 3. 文本分块（Chunking）

将长文档切分成较小的片段，以便更精准地检索相关内容。

In [44]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 创建文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # 每个块最大 500 字符
    chunk_overlap=50,      # 块之间重叠 50 字符（保持上下文连贯）
    length_function=len,
    separators=["\n\n", "\n", "。", "！", "？", ";", ",", " ", ""]
)

# 分割文档
chunks = text_splitter.split_documents(documents)

print(f"✅ 文档已分割为 {len(chunks)} 个文本块")
print(f"\n示例文本块 1:")
print(f"来源: {os.path.basename(chunks[0].metadata['source'])}")
print(f"内容: {chunks[0].page_content}")
print(f"\n示例文本块 2:")
print(f"来源: {os.path.basename(chunks[1].metadata['source'])}")
print(f"内容: {chunks[1].page_content}")

✅ 文档已分割为 15 个文本块

示例文本块 1:
来源: class_roster_cs23-1.txt
内容: # 计算机科学与技术23-1班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-1班

## 班级特色
- 班长: 王天翔（学号：2023217101）
- 学习委员: 李明宇（学号：2023217115）
- 数学最好的学生: 张思远（学号：2023217108）
- 最受欢迎的老师: 数据结构课程的陈建国老师

## 班级花名册（共48人）

示例文本块 2:
来源: class_roster_cs23-1.txt
内容: ### 男生（40人）
1. 王天翔（2023217101）- 共青团员【班长】
2. 赵宇航（2023217102）- 共青团员
3. 李健（2023217103）- 群众
4. 刘浩然（2023217104）- 共青团员
5. 陈俊杰（2023217105）- 共青团员
6. 张伟强（2023217106）- 群众
7. 杨锐（2023217107）- 共青团员
8. 张思远（2023217108）- 共青团员【数学最强】
9. 周文博（2023217109）- 共青团员
10. 吴晨曦（2023217110）- 群众
11. 郑昊（2023217111）- 共青团员
12. 冯宇轩（2023217112）- 共青团员
13. 许志强（2023217113）- 群众
14. 胡磊（2023217114）- 共青团员
15. 李明宇（2023217115）- 共青团员【学习委员】
16. 何峰（2023217116）- 共青团员
17. 马骏（2023217117）- 群众
18. 高翔（2023217118）- 共青团员
19. 徐阳（2023217119）- 共青团员


## 4. 向量化（Embedding）

使用 HuggingFace 的中文 Embedding 模型将文本转换为向量。

In [47]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from modelscope import snapshot_download
import torch
import os

# 使用中文优化的 Embedding 模型（小巧，适合本地运行）
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Embedding 模型将使用设备: {device}")

# 下载 Embedding 模型到本地 models 目录
print("正在下载/加载 Embedding 模型...")
embedding_model_id = 'AI-ModelScope/text2vec-base-chinese'
try:
    embedding_model_dir = snapshot_download(embedding_model_id, cache_dir='./models')
    print(f"Embedding 模型路径: {embedding_model_dir}")
except Exception as e:
    print(f"ModelScope 下载失败，尝试使用默认路径: {e}")
    embedding_model_dir = "shibing624/text2vec-base-chinese"

embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_dir,
    model_kwargs={'device': device},  # 使用 GPU 加速
    encode_kwargs={
        'normalize_embeddings': True,
        'batch_size': 32  # 批量处理，提升速度
    }
)

print("✅ Embedding 模型加载完成")

test_text = "计算机科学与技术23-2班的学习委员"
print(f"\n测试文本: '{test_text}'")
test_vector = embedding_model.embed_query(test_text)
print(f"向量维度: {len(test_vector)}")
print(f"向量前5个元素: {test_vector[:5]}")


Embedding 模型将使用设备: cuda
✅ Embedding 模型加载完成
向量前5个元素: [-0.019558066502213478, -0.0538894347846508, 0.04210386425256729, 0.022998197004199028, 0.02588777244091034]

测试文本: '计算机科学与技术23-2班的学习委员'
✅ Embedding 模型加载完成
向量前5个元素: [-0.019558066502213478, -0.0538894347846508, 0.04210386425256729, 0.022998197004199028, 0.02588777244091034]

测试文本: '计算机科学与技术23-2班的学习委员'


## 5. 构建 FAISS 向量数据库

将所有文本块向量化并存储到 FAISS 索引中。

In [60]:
from langchain_community.vectorstores import FAISS
import time

print("开始构建 FAISS 向量数据库...")
start_time = time.time()

# 创建 FAISS 向量库
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

elapsed = time.time() - start_time
print(f"✅ 向量数据库构建完成！耗时: {elapsed:.2f} 秒")
print(f"索引包含 {vectorstore.index.ntotal} 个向量")

# 保存向量库到本地
vectorstore.save_local("./vector_store")
print("✅ 向量数据库已保存到 ./vector_store")

开始构建 FAISS 向量数据库...
✅ 向量数据库构建完成！耗时: 15.17 秒
索引包含 15 个向量
✅ 向量数据库已保存到 ./vector_store
✅ 向量数据库构建完成！耗时: 15.17 秒
索引包含 15 个向量
✅ 向量数据库已保存到 ./vector_store


## 6. 测试相似度检索

测试检索功能，查看能否找到相关文档片段。

In [53]:
# 测试查询
import time

query = "计算机23-2班的学习委员是谁"

print(f"查询: '{query}'\n")

# 检索相似文档（返回前 3 个最相关的）
start_time = time.time()
results = vectorstore.similarity_search_with_score(query, k=3)
search_time = time.time() - start_time

print(f"⚡ 检索耗时: {search_time*1000:.2f} 毫秒\n")

for i, (doc, score) in enumerate(results):
    print(f"========== 结果 {i+1} (相似度分数: {score:.4f}) ==========")
    print(f"来源: {os.path.basename(doc.metadata['source'])}")
    print(f"内容:\n{doc.page_content}")
    print()

查询: '计算机23-2班的学习委员是谁'

⚡ 检索耗时: 341.70 毫秒

========== 结果 1 (相似度分数: 0.5125) ==========
来源: class_roster_cs23-2.txt
内容:
# 计算机科学与技术23-2班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-2班

## 班级特色
- 班长: 陈浩宇（学号：2023217201）
- 学习委员: 林雨晨（学号：2023217218）
- 数学最好的学生: 周凯（2023217225）
- 最受欢迎的老师: 操作系统课程的李建华老师

## 班级花名册（共52人）

========== 结果 2 (相似度分数: 0.5895) ==========
来源: class_roster_cs23-1.txt
内容:
# 计算机科学与技术23-1班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-1班

## 班级特色
- 班长: 王天翔（学号：2023217101）
- 学习委员: 李明宇（学号：2023217115）
- 数学最好的学生: 张思远（学号：2023217108）
- 最受欢迎的老师: 数据结构课程的陈建国老师

## 班级花名册（共48人）

========== 结果 3 (相似度分数: 0.6718) ==========
来源: class_roster_cs23-3.txt
内容:
# 计算机科学与技术23-3班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-3班

## 班级特色
- 学习委员: 贺鑫帅（学号：2023217373）
- 数学最好的学生: 胡俊贤（学号：2023217363）
- 最受欢迎的老师: 人工智能原理课程的刘雪南老师

## 班级花名册（共50人）

⚡ 检索耗时: 341.70 毫秒

========== 结果 1 (相似度分数: 0.5125) ==========
来源: class_roster_cs23-2.txt
内容:
# 计算机科学与技术23-2班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-2班

#

## 7. 集成本地 LLM 进行 RAG 问答

将检索到的文档片段作为上下文，输入本地 Qwen 模型生成答案。

In [51]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from modelscope import snapshot_download
import torch

print("加载本地 Qwen 模型...")

model_id = 'Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4'
model_dir = snapshot_download(model_id, cache_dir='./models')

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

# 显式指定设备
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"使用设备: {device}")

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map={"": 0} if torch.cuda.is_available() else {"": "cpu"},  # 明确指定 GPU 0 或 CPU
    trust_remote_code=True
)

print("✅ 模型加载完成！")

加载本地 Qwen 模型...


2025-12-14 22:25:09,016 - modelscope - INFO - Creating symbolic link [./models\Qwen\Qwen2.5-7B-Instruct-GPTQ-Int4].
2025-12-14 22:25:09,029 - modelscope - WARNING - Failed to create symbolic link ./models\Qwen\Qwen2.5-7B-Instruct-GPTQ-Int4 for d:\llm_deploy\LLM\LLM_DEPLOY\models\Qwen\Qwen2___5-7B-Instruct-GPTQ-Int4.
2025-12-14 22:25:09,029 - modelscope - WARNING - Failed to create symbolic link ./models\Qwen\Qwen2.5-7B-Instruct-GPTQ-Int4 for d:\llm_deploy\LLM\LLM_DEPLOY\models\Qwen\Qwen2___5-7B-Instruct-GPTQ-Int4.


使用设备: cuda:0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 模型加载完成！


In [54]:
def rag_answer(question, k=3):
    """
    RAG 问答函数
    
    Args:
        question: 用户问题
        k: 检索文档数量
    """
    # 1. 检索相关文档
    retrieved_docs = vectorstore.similarity_search(question, k=k)
    
    # 2. 构建上下文
    context = "\n\n".join([f"【参考资料 {i+1}】\n{doc.page_content}" 
                           for i, doc in enumerate(retrieved_docs)])
    
    # 3. 构建提示词
    prompt = f"""你是一个专业的技术助手。请根据以下参考资料回答问题。如果参考资料中没有相关信息，请明确说明。

{context}

【问题】
{question}

【回答】"""
    
    # 4. 调用模型生成答案
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.8,
        do_sample=True
    )
    
    generated_ids = [
        output_ids[len(input_ids):] 
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    answer = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return answer, retrieved_docs

print("✅ RAG 问答函数已定义")

✅ RAG 问答函数已定义


## 8. RAG 问答测试

In [55]:
# 测试问题 1：私有数据查询
question1 = "计算机科学与技术23-3班的学习委员是谁？他的学号是多少？"

print(f"问题: {question1}\n")
print("检索中...")

answer, docs = rag_answer(question1)

print("\n========== 检索到的参考资料 ==========")
for i, doc in enumerate(docs):
    print(f"\n参考资料 {i+1}: {os.path.basename(doc.metadata['source'])}")
    print(doc.page_content[:200] + "...")

print("\n========== AI 回答 ==========")
print(answer)

问题: 计算机科学与技术23-3班的学习委员是谁？他的学号是多少？

检索中...

========== 检索到的参考资料 ==========

参考资料 1: class_roster_cs23-3.txt
# 计算机科学与技术23-3班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-3班

## 班级特色
- 学习委员: 贺鑫帅（学号：2023217373）
- 数学最好的学生: 胡俊贤（学号：2023217363）
- 最受欢迎的老师: 人工智能原理课程的刘雪南老师

## 班级花名册（共50人）...

参考资料 2: class_roster_cs23-1.txt
# 计算机科学与技术23-1班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-1班

## 班级特色
- 班长: 王天翔（学号：2023217101）
- 学习委员: 李明宇（学号：2023217115）
- 数学最好的学生: 张思远（学号：2023217108）
- 最受欢迎的老师: 数据结构课程的陈建国老师

## 班级花名册（共48人）...

参考资料 3: class_roster_cs23-2.txt
# 计算机科学与技术23-2班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-2班

## 班级特色
- 班长: 陈浩宇（学号：2023217201）
- 学习委员: 林雨晨（学号：2023217218）
- 数学最好的学生: 周凯（2023217225）
- 最受欢迎的老师: 操作系统课程的李建华老师

## 班级花名册（共52人）...

========== AI 回答 ==========
计算机科学与技术23-3班的学习委员是贺鑫帅，他的学号是2023217373。

========== 检索到的参考资料 ==========

参考资料 1: class_roster_cs23-3.txt
# 计算机科学与技术23-3班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-3班

## 班级特色
- 学习委员: 贺鑫帅（学号：2023217373）
- 数学最好的学生: 胡俊贤

In [58]:
# 测试问题 2：私有数据查询
question2 = "计算机23-3班数学最好的学生是谁？班上一共有多少女生？"

print(f"问题: {question2}\n")
answer, docs = rag_answer(question2)

print("\n========== 检索到的参考资料 ==========")
for i, doc in enumerate(docs):
    print(f"\n参考资料 {i+1}: {os.path.basename(doc.metadata['source'])}")
    print(doc.page_content[:200] + "...")

print("\n========== AI 回答 ==========")
print(answer)

问题: 计算机23-3班数学最好的学生是谁？班上一共有多少女生？


========== 检索到的参考资料 ==========

参考资料 1: class_roster_cs23-3.txt
# 计算机科学与技术23-3班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-3班

## 班级特色
- 学习委员: 贺鑫帅（学号：2023217373）
- 数学最好的学生: 胡俊贤（学号：2023217363）
- 最受欢迎的老师: 人工智能原理课程的刘雪南老师

## 班级花名册（共50人）...

参考资料 2: class_roster_cs23-2.txt
# 计算机科学与技术23-2班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-2班

## 班级特色
- 班长: 陈浩宇（学号：2023217201）
- 学习委员: 林雨晨（学号：2023217218）
- 数学最好的学生: 周凯（2023217225）
- 最受欢迎的老师: 操作系统课程的李建华老师

## 班级花名册（共52人）...

参考资料 3: class_roster_cs23-1.txt
# 计算机科学与技术23-1班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-1班

## 班级特色
- 班长: 王天翔（学号：2023217101）
- 学习委员: 李明宇（学号：2023217115）
- 数学最好的学生: 张思远（学号：2023217108）
- 最受欢迎的老师: 数据结构课程的陈建国老师

## 班级花名册（共48人）...

========== AI 回答 ==========
计算机科学与技术23-3班数学最好的学生是胡俊贤（学号：2023217363）。

关于班上女生的人数，参考资料1中并没有提供具体的性别分布信息，因此无法确定计算机科学与技术23-3班有多少女生。

========== 检索到的参考资料 ==========

参考资料 1: class_roster_cs23-3.txt
# 计算机科学与技术23-3班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-3班

In [57]:
# 测试问题 3：私有数据查询
question3 = "计科23-3班上有多少个共青团员？最受欢迎的老师是谁？"

print(f"问题: {question3}\n")
answer, docs = rag_answer(question3)

print("\n========== 检索到的参考资料 ==========")
for i, doc in enumerate(docs):
    print(f"\n参考资料 {i+1}: {os.path.basename(doc.metadata['source'])}")
    print(doc.page_content[:200] + "...")

print("\n========== AI 回答 ==========")
print(answer)

问题: 计科23-3班上有多少个共青团员？最受欢迎的老师是谁？


========== 检索到的参考资料 ==========

参考资料 1: class_roster_cs23-3.txt
# 计算机科学与技术23-3班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-3班

## 班级特色
- 学习委员: 贺鑫帅（学号：2023217373）
- 数学最好的学生: 胡俊贤（学号：2023217363）
- 最受欢迎的老师: 人工智能原理课程的刘雪南老师

## 班级花名册（共50人）...

参考资料 2: class_roster_cs23-1.txt
# 计算机科学与技术23-1班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-1班

## 班级特色
- 班长: 王天翔（学号：2023217101）
- 学习委员: 李明宇（学号：2023217115）
- 数学最好的学生: 张思远（学号：2023217108）
- 最受欢迎的老师: 数据结构课程的陈建国老师

## 班级花名册（共48人）...

参考资料 3: class_roster_cs23-2.txt
# 计算机科学与技术23-2班 班级信息

## 学校信息
学校名称: 合肥工业大学宣城校区
班级名称: 计算机科学与技术23-2班

## 班级特色
- 班长: 陈浩宇（学号：2023217201）
- 学习委员: 林雨晨（学号：2023217218）
- 数学最好的学生: 周凯（2023217225）
- 最受欢迎的老师: 操作系统课程的李建华老师

## 班级花名册（共52人）...

========== AI 回答 ==========
根据提供的参考资料，对于计算机科学与技术23-3班的信息，参考资料中并没有提到共青团员的人数。同时，参考资料中提到最受欢迎的老师是人工智能原理课程的刘雪南老师。因此，可以回答：

- 计科23-3班上的共青团员人数无法从给定的信息中得出。
- 最受欢迎的老师是人工智能原理课程的刘雪南老师。

========== 检索到的参考资料 ==========

参考资料 1: class_roster_cs23-3.txt
# 计算机科学与技术23-3班 班级信息

## 9. 对比：有 RAG vs 无 RAG

对比同一个问题，使用 RAG 和不使用 RAG 的回答差异。

In [59]:
def simple_chat(question):
    """不使用 RAG，直接问模型"""
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.8,
        do_sample=True
    )
    
    generated_ids = [
        output_ids[len(input_ids):] 
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    answer = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return answer

# 对比测试：私有数据问题
test_question = "计算机科学与技术23-1班的学习委员是谁？"

print(f"问题: {test_question}\n")

print("========== 无 RAG 回答 ==========")
no_rag_answer = simple_chat(test_question)
print(no_rag_answer)
print("\n⚠️ 分析: 模型无法准确回答，因为训练数据中不包含这个私有信息")

print("\n========== 有 RAG 回答 ==========")
rag_answer_text, _ = rag_answer(test_question)
print(rag_answer_text)
print("\n✅ 分析: RAG 系统能够从知识库中检索到准确答案")

print("\n========== 对比分析 ==========")
print("✅ RAG 的优势: 能够基于私有知识库回答问题，这些信息在大模型的训练数据中不存在")
print("⚠️ 无 RAG: 只能依赖训练数据，无法回答关于特定班级、公司内部信息等私有数据的问题")
print("\n这就是 RAG 的核心价值：让大模型能够访问和理解你的私有数据！")

问题: 计算机科学与技术23-1班的学习委员是谁？

========== 无 RAG 回答 ==========
很抱歉，作为一个AI，我无法获取到实时的班级信息或特定班级的具体成员信息，包括学习委员是谁。建议你直接咨询你的辅导员或者班长等来获取准确的信息。
⚠️ 模型无法准确回答，因为训练数据中不包含这个私有信息

========== 有 RAG 回答 ==========
很抱歉，作为一个AI，我无法获取到实时的班级信息或特定班级的具体成员信息，包括学习委员是谁。建议你直接咨询你的辅导员或者班长等来获取准确的信息。
⚠️ 模型无法准确回答，因为训练数据中不包含这个私有信息

========== 有 RAG 回答 ==========
计算机科学与技术23-1班的学习委员是李明宇，学号为2023217115。
✅ RAG 系统能够从知识库中检索到准确答案

========== 对比分析 ==========
✅ RAG 的优势: 能够基于私有知识库回答问题，这些信息在大模型的训练数据中不存在
⚠️ 无 RAG: 只能依赖训练数据，无法回答关于特定班级、公司内部信息等私有数据的问题

这就是 RAG 的核心价值：让大模型能够访问和理解你的私有数据！
计算机科学与技术23-1班的学习委员是李明宇，学号为2023217115。
✅ RAG 系统能够从知识库中检索到准确答案

========== 对比分析 ==========
✅ RAG 的优势: 能够基于私有知识库回答问题，这些信息在大模型的训练数据中不存在
⚠️ 无 RAG: 只能依赖训练数据，无法回答关于特定班级、公司内部信息等私有数据的问题

这就是 RAG 的核心价值：让大模型能够访问和理解你的私有数据！


## 10. 总结

### ✅ 已完成内容
本 Notebook 完成了完整的 RAG 系统构建：
1. **知识库处理**：加载并处理班级花名册（计算机科学与技术23-3班，50名学生）
2. **文本分块**：使用 RecursiveCharacterTextSplitter，chunk_size=500，overlap=50
3. **向量化**：使用 text2vec-base-chinese 中文优化模型
4. **FAISS 索引**：构建并持久化向量数据库到 `./vector_store`
5. **相似度检索**：实现 top-k 检索，验证检索质量
6. **RAG 问答**：集成 Qwen2.5-7B 实现增强生成
7. **效果对比**：展示 RAG vs 普通对话的差异

### 🎯 核心优势
- **准确性提升**：基于知识库回答，避免模型幻觉
- **私有数据支持**：能够回答关于班级学生信息等私有数据的问题（大模型训练数据中不存在）
- **可追溯性**：能够明确引用参考资料来源
- **灵活更新**：添加新文档后重新构建索引即可，无需重新训练模型
- **成本低廉**：只需向量化和检索，无需昂贵的 GPU 训练

### 📊 性能指标
- **向量化速度**：取决于文档数量和 Embedding 模型
- **检索延迟**：FAISS 检索毫秒级响应
- **生成延迟**：约 3-4 秒（与模型推理时间相同）

### 🚀 下一步行动
1. **启动 API 服务**：运行 `python rag_server.py` 启动 RAG API
2. **前端交互**：打开 `rag_chat.html` 进行可视化对话
3. **扩展知识库**：
   - 将你的文档添加到 `knowledge_base/sample_docs/`
   - 重新运行本 Notebook 的第 3-5 节构建新索引
4. **优化检索**：
   - 调整 `chunk_size` 和 `chunk_overlap` 参数
   - 尝试混合检索（BM25 + 向量检索）
   - 实现重排序（Re-ranking）提升精度

### 📝 RAG 应用场景
- **企业知识库**：内部文档、规章制度、技术手册
- **教育场景**：班级信息、课程资料、学生档案（如本示例）
- **客户服务**：产品说明书、FAQ、历史工单
- **法律合规**：法规文件、合同模板、案例分析
- **医疗健康**：病历记录、诊疗指南、药品信息

### 💡 扩展建议
- **多模态 RAG**：结合图片、表格等多模态内容
- **对话记忆**：保持多轮对话上下文
- **分布式部署**：使用 Milvus 替代 FAISS 实现分布式
- **实时更新**：监听文档变化，增量更新索引
- **权限控制**：为敏感数据添加访问权限管理